**Tutorial**:
 
1. Download project and install dependencies
2. Preprocess data
3. Segment preprocessed data

**1. Download project and install dependencies**

Download the github project with the following command:

In [ ]:
!git clone https://github.com/HelmholtzAI-Consultants-Munich/AURORA

Install all requirements listed in requirements.txt:

In [ ]:
!cd AURORA
!pip install -r requirements.txt 

In [ ]:
import requests
import os

os.makedirs("Examples", exist_ok="T")

url_label = "https://files-accl.zohopublic.eu/public/workdrive-public/download/bbd0p91ca3309beac4eb69f78a103d686d53a?x-cli-msg=%7B%22isFileOwner%22%3Afalse%2C%22bbd0p91ca3309beac4eb69f78a103d686d53a%22%3A%224uQNLsigiFF-lVsGRB%22%2C%22version%22%3A%221.0%22%2C%224uQNLsigiFF-lVsGRB_user_uuid%22%3A%22%242442093196452756052%22%7D"

myfile = requests.get(url_label)
open("Examples/Nifti.zip", 'wb').write(myfile.content)

!unzip Examples/Nifti.zip -d Examples/Nifti

**2. Preprocess data**

The provided models need coregistered, skullstripped sequences as input.

We recommend BraTS-Toolkit (https://github.com/neuronflow/BraTS-Toolkit) for preprocessing, which covers the entire image analysis workflow prior to tumor segmentation.

BraTS-Toolkit can be downloaded with the following command:

In [ ]:
!pip install BraTS-Toolkit

Please follow the instructione on the Github-page for installation and setup advice.

**3. Segment preprocessed data**

We provide sample data from the ASNR-MICCAI BraTS Brain Metastasis Challenge, which is already preprocessed.

Minimal mode: Segmentation without test-time augmentation with only T1-CE as input.

In [2]:
from lib import single_inference

single_inference(
    t1c_file="Examples/BraTS-MET-00110-000-t1c.nii.gz",
    segmentation_file="your_segmentation_file.nii.gz",
    tta=False,  # optional: whether to use test time augmentations
    verbosity=True,  # optional: verbosity of the output
)

t1: False t1c: True t2: False flair: False
mode: t1c-o
BasicUNet features: (32, 32, 64, 128, 256, 32).
start: 2023-06-10_18-12-37


100%|██████████| 1/1 [00:01<00:00,  1.36s/it]

inputs shape: torch.Size([1, 1, 240, 240, 155])
outputs: torch.Size([1, 2, 240, 240, 155])
data length: 3
outputs shape 0: 1
end: 2023-06-10_18-12-39


There are many possibilities for customization:
- Any of the following combination of sequences can be supplied: 
    - T1-CE + T1 + T2 + T2-FLAIR
    - T1-CE only
    - T1 only
    - T2-FLAIR only
    - T1-CE + T2-FLAIR
    - T1-CE + T1
    - T1-CE + T1 + T2-FLAIR
- Instead of only saving the final output consiting of one file with 2 labels, additional fiules with labels for the whole lesion (metastasis + edema) or the metastasis only can also be saved.
- Test-time augmentation can be enabled. Segmentation with TTA will take 12 times longer than without TTA.


In [2]:
from lib import single_inference

single_inference(
    t1_file="Examples/BraTS-MET-00110-000-t1n.nii.gz",
    t1c_file="Examples/BraTS-MET-00110-000-t1c.nii.gz",
    t2_file="Examples/BraTS-MET-00110-000-t2w.nii.gz",
    fla_file="Examples/BraTS-MET-00110-000-t2f.nii.gz",
    segmentation_file="your_segmentation_file.nii.gz",
    whole_network_outputs_file="your_whole_lesion_file.nii.gz",  # optional: whether to save network outputs for the whole lesion (metastasis + edema)
    metastasis_network_outputs_file="your_metastasis_file.nii.gz",  # optional: whether to save network outputs for the metastasis
    cuda_devices="0",  # optional: which CUDA devices to use
    tta=True,  # optional: whether to use test time augmentations
    sliding_window_batch_size=1,  # optional: adjust to fit your GPU memory, each step requires an additional 2 GB of VRAM, increasing is not recommended for single interference
    workers=8,  # optional: workers for the data laoder
    threshold=0.5,  # optional: where to threshold the network outputs
    sliding_window_overlap=0.5,  # optional: overlap for the sliding window
    model_selection="best",  # optional: choose best or last checkpoint, best is recommended
    verbosity=True,  # optional: verbosity of the output
)

t1: True t1c: True t2: True flair: True
mode: t1-t1c-t2-fla
BasicUNet features: (32, 32, 64, 128, 256, 32).
start: 2023-06-10_18-24-44


100%|██████████| 1/1 [00:18<00:00, 18.30s/it]

13.0
inputs shape: torch.Size([1, 4, 240, 240, 155])
outputs: torch.Size([1, 2, 240, 240, 155])
data length: 6
outputs shape 0: 1


100%|██████████| 1/1 [00:18<00:00, 18.35s/it]

end: 2023-06-10_18-25-02
